## **캡스톤치맥회동 AI 스터디 python**

### 스터디 개요
1. HuggingFace Hub의 imdb 데이터셋을 이용한 전처리 및 텍스트 토큰화
2. 학습, 검증, 테스트 로직 구현
3. 모델 성능 향상

### 스터디로 얻어갈 수 있는 능력
* HuggingFace Hub에서 데이터셋을 불러올 수 있습니다.
* train, validation, test 데이터셋을 분리할 수 있습니다.
* 텍스트 데이터를 전처리하고, 임베딩할 수 있습니다.
* 학습, 검증, 테스트 로직을 익힐 수 있습니다.

> [TODO]에 코드를 채워넣으면 됩니다!

## 1. 데이터셋 불러오기
* 허깅페이스의 imdb 데이터셋을 불러옵니다.
* 참고) imdb는 train, test 데이터셋을 지원합니다.

In [ ]:
import pandas as pd
from datasets import load_dataset  # HuggingFace에서 지원하는 라이브러리입니다.

In [ ]:
train_dataset = load_dataset("imdb", split="train")
test_dataset = load_dataset("imdb", split="test")

In [ ]:
# 데이터셋 길이 확인
print("train dataset 개수:", len(train_dataset))
print("test dataset 개수:", len(test_dataset))

# 데이터셋 모양 확인
print("\ntrain dataset 모양")
print(train_dataset)

print("\ntest dataset 모양")
print(test_dataset)

train dataset 개수: 25000
test dataset 개수: 25000

train dataset 모양
Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

test dataset 모양
Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})


In [ ]:
# Dataset 객체를 전처리 가능한 구조로 변환합니다.
train_df = pd.DataFrame(train_dataset)
test_df = pd.DataFrame(test_dataset)

# 2번에서 train : validation : test 데이터셋을 80 : 10 : 10으로 분할하기 위해, 전체 데이터셋으로 합칩니다.
df = pd.concat([train_df, test_df]).reset_index(drop=True)

# 5개의 텍스트를 출력합니다.
df.head()

,text,label
0,I rented I AM CURIOUS-YELLOW from my video sto...,0
1,"""I Am Curious: Yellow"" is a risible and preten...",0
2,If only to avoid making this type of film in t...,0
3,This film was probably inspired by Godard's Ma...,0
4,"Oh, brother...after hearing about this ridicul...",0


## 2. Train/Val/Test 데이터셋으로 분리하기

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df.text  # df["text"]로 써도 됩니다.
y = df.label

# 1차 분리: train : tmp = 80 : 20
X_train, X_tmp, y_train, y_tmp = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y  # label의 분포를 유지하기 위해 stratify를 명시해줍니다.
)

# 2차 분리: train : val : test = 80 : 10 : 10
X_val, X_test, y_val, y_test = train_test_split(
    X_tmp, y_tmp, test_size=0.5, stratify=y_tmp
)

## 3. 데이터 전처리하기

In [ ]:
import re  # regex: 정규 표현식을 지원하는 라이브러리입니다.

# [TODO 1]
def clean_text(text):
    # [TODO 1-1]. 소문자로 변환합니다.
    text = None

    # [TODO 1-2]. 특수문자, 불필요한 문자를 제거하여, 알파벳과 공백만 남깁니다.
    text = re.sub()

    return text

In [ ]:
# 훈련, 검증, 테스트 데이터셋에 전처리 함수를 적용합니다.
X_train_clean = X_train.apply(clean_text)
X_val_clean = X_val.apply(clean_text)
X_test_clean = X_test.apply(clean_text)

In [ ]:
before_and_after = pd.concat([X_train.head(), X_train_clean.head()], axis=1).reset_index(drop=True)
before_and_after.columns = ["before_text", "after_text"]

# 전처리 전후 텍스트를 5개 출력합니다.
before_and_after

,before_text,after_text
0,Having a close experience with one such patien...,having a close experience with one such patien...
1,This is by far one of the worst movies i have ...,this is by far one of the worst movies i have ...
2,This film is a great disappointment. Director ...,this film is a great disappointment director v...
3,"When I first popped in Happy Birthday to Me, I...",when i first popped in happy birthday to me i ...
4,I signed in just to comment on how awfully stu...,i signed in just to comment on how awfully stu...


## 4. 데이터 임베딩하기
* 임베딩: 텍스트 -> 벡터

### 4-1. tokenizer 구축하기

In [ ]:
from typing import List

In [ ]:
# [TODO 2]
def tokenize(sentence: str) -> List[str]:
    # 문장을 토큰으로 쪼갭니다.
    tokens = re.split(r'\s+|([.,!?]|n\'t|\'\w+)', sentence)

    # [TODO 2-1] 토큰에서 공백을 제거합니다.
    tokens = None

    return tokens

### 4-2. vocabulary 구축하기

In [ ]:
from typing import Dict
from collections import Counter

In [ ]:
# [TODO 3]
def build_vocab(
    sentences: List[str],
    min_freq=5,
    tokenize=tokenize
) -> Dict[str, int]:  # 키(str), 값(int)

    # [TODO 3-1] # 모든 문장의 토큰을 하나의 리스트로 모읍니다.
    # 힌트: 평탄화된 토큰 리스트를 만들기 위해, append 대신 extend를 사용합니다.
    all_tokens = []

    # 각 토큰의 빈도를 셉니다.
    word_counts = Counter(all_tokens)

    # [TODO 3-2] Vocabulary를 구성합니다.
    # 최소 빈도(freq)보다 더 많이 등장한 단어만 id2token에 저장합니다.
    # id2token = ["UNK", "this", "is", "an", "apple", ...]
    # [UNK] 토큰의 id는 0으로 처리합니다.
    id2token = ["UNK"]

    # [TODO 3-3] id2token을 기반으로 token2id를 구축합니다.
    # token2id = {"UNK": 100, "this": 50, "is": 25, ...}
    token2id = {}

    return token2id

In [ ]:
# 훈련 데이터셋의 텍스트로 사전을 구축합니다.
token2id = build_vocab(X_train_clean.to_list())

### 4-3. 임베딩

In [ ]:
# [TODO 4]
def encode(
    sentence: str,
    tokenize=tokenize,
    token2id=token2id,
    max_seq_len=128
) -> List[int]:
    # 주어진 tokenize 함수를 사용하여 문장의 단어 목록으로 변환합니다.
    tokens = tokenize(sentence)

    # [TODO 4-1] 토큰을 ID로 변환해, 리스트로 저장합니다.
    token_ids = []

    if len(token_ids) < max_seq_len:
        # [TODO 4-2] 시퀀스의 길이를 맞추기 위해 0 패딩을 적용합니다.
        pass
    else:
        # [TODO 4-3] 시퀀스가 너무 길 때, 지정된 최대 길이에 맞게 잘라냅니다.
        pass

    return token_ids

In [ ]:
# train, val, test 데이터셋에서 setence를 모두 토큰으로 변환합니다.
X_train_vec = [encode(sentence) for sentence in X_train_clean]
X_val_vec = [encode(sentence) for sentence in X_val_clean]
X_test_vec = [encode(sentence) for sentence in X_test_clean]

In [ ]:
print("임베딩 전:", X_train_clean.iloc[5])
print("임베딩 후:", X_train_vec[5])

임베딩 전: this is my favourite indian movie of all time it is comic genius salman khan is hilarious but amir khan steals the show with his witty dialogue karisma kapoors outfits tell a story of their own  makes you wonder if the stylist deliberately made her wear some of the clothes just to make the movie funnier at one point she looks like shes wearing a nappy andaz apna apna is the only comedy genre movie to make me laugh from the beginning till the very end there is not one dull moment every scene is hilarious even the songs and dance moves will have you in stitches of laughter i especially loved the scene in which amar amir khan regains his memory ive seen this movie so many times ive lost count and im so glad to say that this time bollywood can take all the credit for this fantastic movie as far as i know aaa it is not a replicate of a hollywood movie thank god overall i recommend this movie to anyone who understands hindi urdu and loves good comedybr br watch it youll love it
임베딩 후:

## 5. 학습

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(random_state=42)

# [TODO 5] X_train_vec, y_train을 기반으로 학습을 진행합니다.
pass

print("학습을 완료했습니다.")

학습을 완료했습니다.


## 6. 검증

In [ ]:
from sklearn.metrics import accuracy_score

# [TODO 6] X_val_vec, y_val을 기반으로 검증을 진행합니다.
y_val_pred=None

val_accuracy = accuracy_score(y_val, y_val_pred)

print("검증 데이터 정확도:", val_accuracy)

검증 데이터 정확도: 0.5354


## 7. 테스트

In [ ]:
from sklearn.metrics import confusion_matrix

y_test_pred = model.predict(X_test_vec)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("테스트 데이터 정확도:", test_accuracy)

conf_matrix = confusion_matrix(y_test, y_test_pred)

# 긍정(1)과 부정(0) 레이블의 의미를 해석할 수 있도록 출력합니다.
# TN (True Negative): 실제 0을 0으로 맞힘
# FP (False Positive): 실제 0을 1로 틀림 (오탐)
# FN (False Negative): 실제 1을 0으로 틀림 (미탐)
# TP (True Positive): 실제 1을 1로 맞힘
print(f"TN: {conf_matrix[0, 0]}, FP: {conf_matrix[0, 1]}")
print(f"FN: {conf_matrix[1, 0]}, TP: {conf_matrix[1, 1]}")

테스트 데이터 정확도: 0.5258
TN: 1479, FP: 1021
FN: 1350, TP: 1150
